In [1]:
import lightgbm as lgbm
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import matplotlib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pickle
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from scipy import stats
from scipy.stats import zscore
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold as SKF
from sklearn import metrics
# from fancyimpute import *

from utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

dtype = load_obj('dict_dtype')
my_dict = load_obj('my_dict')

In [2]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)
test = pd.read_csv("atec_anti_fraud_test_b.csv",parse_dates=['date'], dtype = dtype)

data = process_dates(data, 'date')
test = process_dates(test, 'date')

In [3]:
# set the label of unlabeled data to 1
data.loc[data['label']==-1,'label'] = 1

#data = data[data['label']!=-1]

# sort the data with date
data.sort_values('date',inplace=True)

print(data.shape)

(994731, 304)


### xgboost without 5-fold (local: 0.409, lb: 0.309)

In [ ]:
# set up the parameters
params = {'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss','auc']
params["scale_pos_weight"] = 2
params["subsample"] = 0.7
params["colsample_bytree "] = 0.8
num_rounds = 10000
early_stopping_rounds = 50

# set up the random seed for testing
params["seed"] = 6

xgtrain = xgb.DMatrix(data.drop(['id','date','label','Day'],axis=1).values, data.label.values)

evallist = [(xgtrain, 'train'), (xgvalid, 'valid')]
bst = xgb.train(params,
                xgtrain,
                num_rounds,
                evallist,
                # feval=my_score3,
                early_stopping_rounds=early_stopping_rounds)
# save the model
bst.save_model('model_log/single_1/0001.model')

cv_pred += bst.predict(xgtest)
valid_pred += bst.predict(xgvalid, ntree_limit=bst.best_iteration)

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))
pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0003.csv', index=False)

### 5-fold time dependent cv train on first 80% data xgboost (local:0.42, lb:0.3383)

In [ ]:
# use the last 20% data as validation set
valid = data.iloc[data.shape[0]*4//5:,:]
data = data.iloc[:data.shape[0]*4//5,:]

print(data.shape[0] + valid.shape[0], data.shape[1])

test_id = test.id
cv_pred = np.zeros(len(test_id))
xgtest = xgb.DMatrix(test.drop(['id','date','Day'],axis=1).values)

valid_pred = np.zeros(valid.shape[0])
valid_id = valid.id
xgvalid = xgb.DMatrix(valid.drop(['id','date','label','Day'],axis=1).values, valid.label.values)

In [ ]:
for i in range(0,9,2):
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    # print(day_target)
    train = data[data.Day.isin(day_target)]
    
    # set up the parameters
    params = {'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
    params['nthread'] = 4
    params['eval_metric'] = ['logloss','auc']
    params["scale_pos_weight"] = 2
    params["subsample"] = 0.7
    params["colsample_bytree "] = 0.8
    num_rounds = 10000
    early_stopping_rounds = 50

    # set up the random seed for testing
    params["seed"] = 6
    
    xgtrain = xgb.DMatrix(train.drop(['id','date','label','Day'],axis=1).values, train.label.values)
    
    evallist = [(xgtrain, 'train'), (xgvalid, 'valid')]
    bst = xgb.train(params,
                    xgtrain,
                    num_rounds,
                    evallist,
                    #feval=my_score3,
                    early_stopping_rounds=early_stopping_rounds)
    # save the model
    bst.save_model('model_log/multi_2/part_{}.model'.format(i//2))
    
    valid_pred += bst.predict(xgvalid, ntree_limit=bst.best_iteration)
    cv_pred += bst.predict(xgtest, ntree_limit=bst.best_iteration)

valid_pred /= 5    
cv_pred /= 5

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0002.csv', index=False)

### oversampling + 5-fold time dependent cv train on first 80% data xgboost (local: 0.418)

In [ ]:
# use the last 20% data as validation set
valid = data.iloc[data.shape[0]*4//5:,:]
data = data.iloc[:data.shape[0]*4//5,:]
# positive = data[data['label']==1]
# data = data[data['label']==0]

print(data.shape[0] + valid.shape[0], data.shape[1])

test_id = test.id
cv_pred = np.zeros(len(test_id))
xgtest = xgb.DMatrix(test.drop(['id','date','Day'],axis=1).values)

valid_pred = np.zeros(valid.shape[0])
valid_id = valid.id
xgvalid = xgb.DMatrix(valid.drop(['id','date','label','Day'],axis=1).values, valid.label.values)

In [ ]:
for i in range(0,9,2):
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    
    train = data[data.Day.isin(day_target)]
#     train = train.append(positive)
    
    # set up the parameters
    params = {'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
    params['nthread'] = 4
    params['eval_metric'] = ['logloss','auc']
    params["scale_pos_weight"] = 2
#     params["subsample"] = 0.7
    params["colsample_bytree "] = 0.8
    num_rounds = 10000
    early_stopping_rounds = 50

    # set up the random seed for testing
    params["seed"] = 6
    
    xgtrain = xgb.DMatrix(train.drop(['id','date','label','Day'],axis=1).values, train.label.values)
    
    evallist = [(xgtrain, 'train'), (xgvalid, 'valid')]
    bst = xgb.train(params,
                    xgtrain,
                    num_rounds,
                    evallist,
                    #feval=my_score3,
                    early_stopping_rounds=early_stopping_rounds)
    # save the model
    bst.save_model('model_log/multi_2/part_{}.model'.format(i//2))
    
    valid_pred += bst.predict(xgvalid, ntree_limit=bst.best_iteration)
    cv_pred += bst.predict(xgtest, ntree_limit=bst.best_iteration)

valid_pred /= 5    
cv_pred /= 5

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0007.csv', index=False)

### Train on the whole dataset xgboost 5-fold (no local, lb:0.3394)

In [ ]:
test_id = test.id
cv_pred = np.zeros(len(test_id))
xgtest = xgb.DMatrix(test.drop(['id','date','Day'],axis=1).values)

In [ ]:
tuned_rounds = [226, 327, 136, 418, 289]
for i in range(0,9,2):
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    # print(day_target)
    train = data[data.Day.isin(day_target)]
    
    # set up the parameters
    params = {'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic'}
    params['nthread'] = 4
    params['eval_metric'] = ['logloss','auc']
    params["scale_pos_weight"] = 2
    params["subsample"] = 0.7
    params["colsample_bytree "] = 0.8
    num_rounds = tuned_rounds[i//2]
    early_stopping_rounds = 50

    # set up the random seed for testing
    params["seed"] = 6
    
    xgtrain = xgb.DMatrix(train.drop(['id','date','label','Day'],axis=1).values, train.label.values)
    
    evallist = [(xgtrain, 'train')]
    bst = xgb.train(params,
                    xgtrain,
                    num_rounds,
                    evallist,
                    #feval=my_score3,
                    early_stopping_rounds=early_stopping_rounds)
    # save the model
    bst.save_model('model_log/multi_3/part_{}.model'.format(i//2))
    
    cv_pred += bst.predict(xgtest, ntree_limit=bst.best_iteration)

cv_pred /= 5
pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0002.csv', index=False)

### oversampling + 5-fold time dependent cv train on first 80% data lightGBM (local: 0006.csv->0.4155) 

In [ ]:
# use the last 20% data as validation set
valid = data.iloc[data.shape[0]*4//5:,:]
data = data.iloc[:data.shape[0]*4//5,:]
# positive = data[data['label']==1]
# data = data[data['label']==0]

# print(data.shape[0] + valid.shape[0], data.shape[1])

test_id = test.id
cv_pred = np.zeros(len(test_id))

valid_pred = np.zeros(valid.shape[0])
valid_id = valid.id

In [ ]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          # "is_unbalance": True,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9,
          "scale_pos_weight": 10
          }
for i in range(0,9,2):
    params['seed'] = i
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    
    train = data[data.Day.isin(day_target)]
#     train = train.append(positive)
    
    dtrain = lgbm.Dataset(train.drop(['id','date','label','Day'],axis=1).values, train.label.values)
    dvalid = lgbm.Dataset(valid.drop(['id','date','label','Day'],axis=1).values, valid.label.values, reference=dtrain)
    bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=my_score2, verbose_eval=100,
                     early_stopping_rounds=100)
    # make the predictions
    cv_pred += bst.predict(test.drop(['id','date','Day'],axis=1).values, num_iteration=bst.best_iteration)
    valid_pred += bst.predict(valid.drop(['id','date','label','Day'],axis=1).values, num_iteration=bst.best_iteration)

valid_pred /= 5    
cv_pred /= 5

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0008.csv', index=False)

### 5-fold time dependent cv train on whole data lightGBM ( no local, lb: 0.33) 

In [4]:
# use the last 20% data as validation set
# valid = data.iloc[data.shape[0]*4//5:,:]
# data = data.iloc[:data.shape[0]*4//5,:]

# print(data.shape[0] + valid.shape[0], data.shape[1])

test_id = test.id
cv_pred = np.zeros(len(test_id))

# valid_pred = np.zeros(valid.shape[0])
# valid_id = valid.id

In [5]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_rounds = [166, 125, 266, 159, 266]
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 1,
          #"metric": 'auc'
          }
for i in range(0,9,2):
    params['seed'] = i
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    # print(day_target)
    train = data[data.Day.isin(day_target)]
    
    dtrain = lgbm.Dataset(train.drop(['id','date','label','Day'],axis=1).values, train.label.values)
    bst = lgbm.train(params, dtrain, num_boost_rounds[i//2],\
                     #valid_sets=dvalid,\
                     feval=my_score2,\
                     verbose_eval=100)
    # make the predictions
    cv_pred += bst.predict(test.drop(['id','date','Day'],axis=1).values, num_iteration=bst.best_iteration)

cv_pred /= 5

# print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0011.csv', index=False)

### 5-fold time dependent cv train on first 80% data lightGBM (local: 0.415, lb: 0.3413) 

#### training process

In [4]:
# use the last 20% data as validation set
valid = data.iloc[data.shape[0]*4//5:,:]
data = data.iloc[:data.shape[0]*4//5,:]

# print(data.shape[0] + valid.shape[0], data.shape[1])

test_id = test.id
cv_pred = np.zeros(len(test_id))

valid_pred = np.zeros(valid.shape[0])
#valid_id = valid.id

In [5]:
print(data.date.max(), valid.date.max())

2017-10-24 00:00:00 2017-11-05 00:00:00


In [8]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9,
          #"metric": 'auc'
          }
for i in range(0,9,2):
    params['seed'] = i
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    # print(day_target)
    train = data[data.Day.isin(day_target)]
    
    dtrain = lgbm.Dataset(train.drop(['id','date','label','Day'],axis=1).values, train.label.values)
    dvalid = lgbm.Dataset(valid.drop(['id','date','label','Day'],axis=1).values, valid.label.values, reference=dtrain)
    bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid,\
                     feval=my_score2,\
                     verbose_eval=100,
                     early_stopping_rounds=100)
    # make the predictions
    cv_pred += bst.predict(test.drop(['id','date','Day'],axis=1).values, num_iteration=bst.best_iteration)
    valid_pred += bst.predict(valid.drop(['id','date','label','Day'],axis=1).values, num_iteration=bst.best_iteration)

valid_pred /= 5    
cv_pred /= 5

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

# pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0009.csv', index=False)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.0391239	valid_0's score: 0.408669
[200]	valid_0's binary_logloss: 0.0382933	valid_0's score: 0.409345
Early stopping, best iteration is:
[166]	valid_0's binary_logloss: 0.0383707	valid_0's score: 0.4139
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.0393144	valid_0's score: 0.403379
[200]	valid_0's binary_logloss: 0.0386405	valid_0's score: 0.40047
Early stopping, best iteration is:
[125]	valid_0's binary_logloss: 0.0388871	valid_0's score: 0.408022
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.0385751	valid_0's score: 0.411784
[200]	valid_0's binary_logloss: 0.0378128	valid_0's score: 0.414575
[300]	valid_0's binary_logloss: 0.0377245	valid_0's score: 0.414487
Early stopping, best iteration is:
[266]	valid_0's binary_logloss: 0.0376993	valid_0's score: 0.417896
Training until validation sco

#### analysis on false negatives

In [9]:
train_cate = pd.read_csv("obj/id_cate_train.csv")
valid_pred = pd.DataFrame({'id': valid.id, 'score': valid_pred, 'label':  valid.label})
valid_pred = valid_pred.merge(train_cate, how='inner', on='id')
valid_pred.head()

,id,score,label,cate
0,96c11ea394631bcc2ae77455cd10cc3dc147f78632921e...,0.000395,0,2
1,eafae698fc60c7a192a1d13074215a25ac24a2e47e12c2...,0.000187,0,0
2,02dc77c122c4036ccab9f003aa760865b6f79632f930e5...,0.000227,0,2
3,e8c3a7fbd718708534a436d217a76c86ac2e0683b65ae1...,0.003345,0,7
4,d89c7b05241c510b3b24371fccd7be3667099f89f4a2ec...,0.248242,1,7


In [11]:
percent = valid_pred.label.values.sum()/valid_pred.shape[0]

threshold = valid_pred.score.quantile(1-percent)
valid_pred['pred_label'] = valid_pred.score.apply(lambda x: int(x>threshold))
valid_pred['isCorrect'] = valid_pred['label']==valid_pred['pred_label']
valid_pred['isCorrect'] = valid_pred['isCorrect'].astype('int')
valid_pred.head()

0.01710505813106003

In [24]:
valid_pred[valid_pred['isCorrect']==0].groupby('cate').id.count()

cate
0      935
1       29
2      431
3        4
4        1
5       81
6        3
7     1303
8       12
9       51
10      15
11      16
13       1
14      18
18      10
21       1
Name: id, dtype: int64

In [25]:
valid_pred.groupby('cate').id.count()

cate
0     86839
1      5361
2     41173
3      9274
4       319
5      9410
6      3603
7     24768
8     10426
9      1549
10     1934
11      602
12       27
13      196
14     3010
15       41
16      120
17       37
18      151
19        4
20       55
21        2
22        4
23       16
24       11
25        3
26        3
28        6
30        1
31        1
32        1
Name: id, dtype: int64

In [27]:
valid_pred[valid_pred['label']==1][valid_pred['isCorrect']==0].groupby('cate').id.count()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


cate
0     396
1      11
2     160
3       4
5      64
6       3
7     738
8      12
9      28
10     12
11     14
13      1
14     12
18      1
Name: id, dtype: int64

In [28]:
valid_pred[valid_pred['label']==1].groupby('cate').id.count()

cate
0     1300
1       21
2      486
3        4
5       64
6        3
7     1415
8       19
9       35
10      15
11      16
13       9
14      15
18       1
Name: id, dtype: int64

In [29]:
display_all(valid_pred[valid_pred['label']==1][valid_pred['isCorrect']==0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,id,score,label,cate,pred_label,isCorrect
4,d89c7b05241c510b3b24371fccd7be3667099f89f4a2ec...,0.248242,1,7,0,0
124,694cf434f6355cfc7856a1568ac3ff488609fadcb5fe8a...,0.273763,1,7,0,0
244,40eb54700d247e26fb28f56a18ce9b6aa5254b7f3d41a4...,0.050994,1,14,0,0
254,d9112cda7ae661fd2dad10a5f929e61d5fa17fdef124bb...,0.002252,1,7,0,0
293,30821797d2f24d7f1dc2dcce21cc3c96be719928abd76e...,0.226805,1,7,0,0
392,27f2a4d7eaeb69a2d45f8a5a143d48677afd456a9dcffa...,0.024728,1,0,0,0
766,678e3ef9449e3d53153660ea0d9411760e2cd83390a4ba...,0.036332,1,7,0,0
875,9333010380982d928c22dc3493d0278612166f010695bd...,0.086421,1,7,0,0
1005,34f99ed9074858c2f5ae38f2e7df27619c6254144f426a...,0.181416,1,0,0,0
1085,52dafb68c5f8c8e36730f1705843422e2c418ec5867df9...,0.174863,1,14,0,0


#### add weights depending on cate (failed)

In [31]:
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)
data = data.merge(train_cate, how='inner', on='id')
data = data[['id', 'label', 'cate']]

tmp1 = data[data['label']==0].groupby('cate').id.count()
tmp2 = data[data['label']==1].groupby('cate').id.count()

df_tmp = pd.concat([tmp1,tmp2],axis=1).fillna(0)
df_tmp.columns = ['total', 'num_positive']

df_tmp['cate'] = df_tmp.index
df_tmp['weight'] = df_tmp['num_positive']/df_tmp['total']
df_tmp['weight'] = df_tmp['weight']/df_tmp['weight'].sum()

pred_valid_withWeight = pd.merge(valid_pred, df_tmp, how='left', on='cate')
pred_valid_withWeight['weighted_score'] = pred_valid_withWeight.score.multiply(pred_valid_withWeight.weight)

print('Final score of validation is {}'.format(my_score1(valid.label.values, pred_valid_withWeight.weighted_score.values)))

In [105]:
pred_valid_withWeight['weighted_score'] = pred_valid_withWeight.score

pred_valid_withWeight.loc[pred_valid_withWeight.cate==7,'weighted_score'] = pred_valid_withWeight.loc[pred_valid_withWeight.cate==7,'weighted_score']
pred_valid_withWeight.loc[pred_valid_withWeight.cate==0,'weighted_score'] = pred_valid_withWeight.loc[pred_valid_withWeight.cate==0,'weighted_score']

In [106]:
print('Final score of validation is {}'.format(my_score1(valid.label.values, pred_valid_withWeight.weighted_score.values)))

Final score of validation is 0.268939171319424


#### just focus on category 7

In [3]:
train_cate = pd.read_csv("obj/id_cate_train.csv")
data = pd.read_csv("atec_anti_fraud_train.csv",parse_dates=['date'], dtype = dtype)
data.loc[data['label']==-1,'label'] = 1
data = process_dates(data, 'date')
data = data.merge(train_cate, how='inner', on='id')
data.sort_values('date',inplace=True)

In [4]:
temp = data[data['cate']==7].head(1)\
    .drop(['id','label','date', 'cate', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start'], axis=1).values[0]
temp_list = list()
for j in range(297):
    if np.isnan(temp[j]):
        temp_list.append(0)
    else:
        temp_list.append(1)

feature_list = list()
for i in range(297):
    if temp_list[i]:
        feature_list.append('f{}'.format(i+1))

In [5]:
valid = data.iloc[data.shape[0]*4//5:,:][['id','label','date', 'cate', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start']+feature_list]
data = data.iloc[:data.shape[0]*4//5,:][['id','label','date', 'cate', 'Day', 'Dayofweek', 'Is_month_end', 'Is_month_start']+feature_list]
valid = valid[valid['cate']==7]
#data = data[data['cate']==7]

In [6]:
# initialize the predictions
valid_pred = np.zeros(valid.shape[0])

In [7]:
# set up the parameters
params = {'max_depth': 5, 'eta': 0.009, 'silent': 1, 'objective': 'binary:logistic'}
params['nthread'] = 4
params['eval_metric'] = ['logloss','auc']
params["scale_pos_weight"] = 2
params["subsample"] = 0.5
params["colsample_bytree "] = 0.5
num_rounds = 10000
early_stopping_rounds = 200

# set up the random seed for testing
params["seed"] = 6

xgtrain = xgb.DMatrix(data.drop(['id','date','label','Day','cate'],axis=1).values, data.label.values)
xgvalid = xgb.DMatrix(valid.drop(['id','date','label','Day','cate'],axis=1).values, valid.label.values)

evallist = [(xgtrain, 'train'), (xgvalid, 'valid')]
bst = xgb.train(params,
                xgtrain,
                num_rounds,
                evallist,
                # feval=my_score3,
                early_stopping_rounds=early_stopping_rounds)
# # save the model
# bst.save_model('model_log/single_1/0001.model')

valid_pred += bst.predict(xgvalid, ntree_limit=bst.best_iteration)

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))
# pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0003.csv', index=False)

[0]	train-logloss:0.684848	train-auc:0.874014	valid-logloss:0.686086	valid-auc:0.808284
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[1]	train-logloss:0.676695	train-auc:0.874631	valid-logloss:0.679138	valid-auc:0.811398
[2]	train-logloss:0.668684	train-auc:0.874739	valid-logloss:0.672319	valid-auc:0.814056
[3]	train-logloss:0.660814	train-auc:0.886912	valid-logloss:0.665545	valid-auc:0.822289
[4]	train-logloss:0.653086	train-auc:0.886881	valid-logloss:0.658985	valid-auc:0.82283
[5]	train-logloss:0.645485	train-auc:0.887078	valid-logloss:0.652497	valid-auc:0.823653
[6]	train-logloss:0.638013	train-auc:0.888848	valid-logloss:0.646004	valid-auc:0.826896
[7]	train-logloss:0.63066	train-auc:0.888533	valid-logloss:0.639737	valid-auc:0.825335
[8]	train-logloss:0.623428	train-auc:0.888684	valid-logloss:0.633567	valid-auc:0.825992
[9]	train-logloss:0.616321	train-auc:0.888851	valid-logloss:0.6274

[91]	train-logloss:0.277893	train-auc:0.906023	valid-logloss:0.337056	valid-auc:0.838845
[92]	train-logloss:0.27558	train-auc:0.906047	valid-logloss:0.335057	valid-auc:0.838922
[93]	train-logloss:0.273293	train-auc:0.906083	valid-logloss:0.333051	valid-auc:0.839414
[94]	train-logloss:0.27103	train-auc:0.906098	valid-logloss:0.331139	valid-auc:0.83948
[95]	train-logloss:0.268795	train-auc:0.906127	valid-logloss:0.329214	valid-auc:0.839313
[96]	train-logloss:0.266585	train-auc:0.906195	valid-logloss:0.327246	valid-auc:0.840025
[97]	train-logloss:0.264398	train-auc:0.906203	valid-logloss:0.325385	valid-auc:0.839863
[98]	train-logloss:0.26224	train-auc:0.906228	valid-logloss:0.323552	valid-auc:0.839796
[99]	train-logloss:0.260108	train-auc:0.906195	valid-logloss:0.321776	valid-auc:0.839733
[100]	train-logloss:0.257999	train-auc:0.906241	valid-logloss:0.319925	valid-auc:0.839806
[101]	train-logloss:0.255913	train-auc:0.906239	valid-logloss:0.318153	valid-auc:0.839955
[102]	train-logloss:0.2

[183]	train-logloss:0.143453	train-auc:0.912499	valid-logloss:0.223325	valid-auc:0.855414
[184]	train-logloss:0.142598	train-auc:0.912486	valid-logloss:0.222619	valid-auc:0.855394
[185]	train-logloss:0.14175	train-auc:0.912462	valid-logloss:0.221929	valid-auc:0.855253
[186]	train-logloss:0.140908	train-auc:0.912464	valid-logloss:0.221225	valid-auc:0.85524
[187]	train-logloss:0.140071	train-auc:0.912506	valid-logloss:0.220547	valid-auc:0.855474
[188]	train-logloss:0.139243	train-auc:0.913772	valid-logloss:0.219901	valid-auc:0.855586
[189]	train-logloss:0.138425	train-auc:0.91384	valid-logloss:0.219267	valid-auc:0.855956
[190]	train-logloss:0.137615	train-auc:0.913895	valid-logloss:0.218609	valid-auc:0.856159
[191]	train-logloss:0.136816	train-auc:0.913942	valid-logloss:0.217961	valid-auc:0.856696
[192]	train-logloss:0.136018	train-auc:0.914048	valid-logloss:0.217304	valid-auc:0.856934
[193]	train-logloss:0.135236	train-auc:0.914067	valid-logloss:0.216693	valid-auc:0.857107
[194]	train-l

[275]	train-logloss:0.091314	train-auc:0.923783	valid-logloss:0.181506	valid-auc:0.865044
[276]	train-logloss:0.090969	train-auc:0.923821	valid-logloss:0.181238	valid-auc:0.865142
[277]	train-logloss:0.090624	train-auc:0.923847	valid-logloss:0.180972	valid-auc:0.865244
[278]	train-logloss:0.09028	train-auc:0.923934	valid-logloss:0.18069	valid-auc:0.865322
[279]	train-logloss:0.089941	train-auc:0.923999	valid-logloss:0.18041	valid-auc:0.865338
[280]	train-logloss:0.089607	train-auc:0.923995	valid-logloss:0.180152	valid-auc:0.865482
[281]	train-logloss:0.089276	train-auc:0.923993	valid-logloss:0.179899	valid-auc:0.86566
[282]	train-logloss:0.088948	train-auc:0.923986	valid-logloss:0.179622	valid-auc:0.866315
[283]	train-logloss:0.088623	train-auc:0.92402	valid-logloss:0.179352	valid-auc:0.86635
[284]	train-logloss:0.0883	train-auc:0.924059	valid-logloss:0.179109	valid-auc:0.866501
[285]	train-logloss:0.087982	train-auc:0.924109	valid-logloss:0.178872	valid-auc:0.866502
[286]	train-loglos

[367]	train-logloss:0.069831	train-auc:0.930787	valid-logloss:0.164873	valid-auc:0.869907
[368]	train-logloss:0.069681	train-auc:0.930827	valid-logloss:0.164769	valid-auc:0.869986
[369]	train-logloss:0.069534	train-auc:0.930906	valid-logloss:0.164641	valid-auc:0.87012
[370]	train-logloss:0.06939	train-auc:0.930969	valid-logloss:0.164536	valid-auc:0.87019
[371]	train-logloss:0.069247	train-auc:0.931023	valid-logloss:0.164425	valid-auc:0.870177
[372]	train-logloss:0.069106	train-auc:0.931059	valid-logloss:0.164326	valid-auc:0.870238
[373]	train-logloss:0.068967	train-auc:0.931121	valid-logloss:0.164186	valid-auc:0.870425
[374]	train-logloss:0.068827	train-auc:0.931199	valid-logloss:0.164093	valid-auc:0.870434
[375]	train-logloss:0.068692	train-auc:0.93124	valid-logloss:0.163995	valid-auc:0.870571
[376]	train-logloss:0.068557	train-auc:0.931322	valid-logloss:0.163926	valid-auc:0.87049
[377]	train-logloss:0.068423	train-auc:0.931676	valid-logloss:0.163805	valid-auc:0.870587
[378]	train-log

...

[3195]	train-logloss:0.045135	train-auc:0.963292	valid-logloss:0.149431	valid-auc:0.894161
[3196]	train-logloss:0.045134	train-auc:0.963297	valid-logloss:0.149448	valid-auc:0.894136
[3197]	train-logloss:0.04513	train-auc:0.963308	valid-logloss:0.14945	valid-auc:0.894144
[3198]	train-logloss:0.045127	train-auc:0.963312	valid-logloss:0.149435	valid-auc:0.894153
[3199]	train-logloss:0.045126	train-auc:0.963313	valid-logloss:0.149437	valid-auc:0.894152
[3200]	train-logloss:0.045125	train-auc:0.963315	valid-logloss:0.149433	valid-auc:0.894158
[3201]	train-logloss:0.045123	train-auc:0.96332	valid-logloss:0.149428	valid-auc:0.894163
[3202]	train-logloss:0.045121	train-auc:0.963325	valid-logloss:0.149422	valid-auc:0.894166
[3203]	train-logloss:0.045117	train-auc:0.963329	valid-logloss:0.149412	valid-auc:0.894198
[3204]	train-logloss:0.045114	train-auc:0.963335	valid-logloss:0.149418	valid-auc:0.894194
[3205]	train-logloss:0.045113	train-auc:0.963341	valid-logloss:0.149408	valid-auc:0.894241
[3

[3286]	train-logloss:0.044966	train-auc:0.96366	valid-logloss:0.149442	valid-auc:0.894164
[3287]	train-logloss:0.044964	train-auc:0.963666	valid-logloss:0.149437	valid-auc:0.894169
[3288]	train-logloss:0.044963	train-auc:0.96367	valid-logloss:0.149441	valid-auc:0.89417
[3289]	train-logloss:0.044961	train-auc:0.963672	valid-logloss:0.14944	valid-auc:0.894171
[3290]	train-logloss:0.04496	train-auc:0.963677	valid-logloss:0.149446	valid-auc:0.894167
[3291]	train-logloss:0.044957	train-auc:0.963681	valid-logloss:0.149452	valid-auc:0.894148
[3292]	train-logloss:0.044954	train-auc:0.963686	valid-logloss:0.149448	valid-auc:0.894143
[3293]	train-logloss:0.044952	train-auc:0.963688	valid-logloss:0.149452	valid-auc:0.894146
[3294]	train-logloss:0.044951	train-auc:0.963691	valid-logloss:0.149453	valid-auc:0.894149
[3295]	train-logloss:0.044946	train-auc:0.963695	valid-logloss:0.149443	valid-auc:0.89415
[3296]	train-logloss:0.044944	train-auc:0.963697	valid-logloss:0.149443	valid-auc:0.894147
[3297

[3377]	train-logloss:0.044791	train-auc:0.964028	valid-logloss:0.14947	valid-auc:0.894233
[3378]	train-logloss:0.044789	train-auc:0.964033	valid-logloss:0.149474	valid-auc:0.894231
[3379]	train-logloss:0.044788	train-auc:0.964035	valid-logloss:0.149476	valid-auc:0.894226
[3380]	train-logloss:0.044784	train-auc:0.964039	valid-logloss:0.149468	valid-auc:0.894273
[3381]	train-logloss:0.044783	train-auc:0.964043	valid-logloss:0.149465	valid-auc:0.894268
[3382]	train-logloss:0.04478	train-auc:0.964047	valid-logloss:0.149439	valid-auc:0.894333
[3383]	train-logloss:0.04478	train-auc:0.964051	valid-logloss:0.149441	valid-auc:0.894332
[3384]	train-logloss:0.044777	train-auc:0.964053	valid-logloss:0.149447	valid-auc:0.894312
[3385]	train-logloss:0.044774	train-auc:0.96406	valid-logloss:0.149449	valid-auc:0.894314
[3386]	train-logloss:0.044773	train-auc:0.964063	valid-logloss:0.149447	valid-auc:0.894322
[3387]	train-logloss:0.044771	train-auc:0.964067	valid-logloss:0.149448	valid-auc:0.89432
[338

[3468]	train-logloss:0.044631	train-auc:0.96437	valid-logloss:0.1495	valid-auc:0.894374
[3469]	train-logloss:0.044629	train-auc:0.964377	valid-logloss:0.14951	valid-auc:0.894364
[3470]	train-logloss:0.044627	train-auc:0.964381	valid-logloss:0.149527	valid-auc:0.894344
[3471]	train-logloss:0.044628	train-auc:0.964382	valid-logloss:0.149523	valid-auc:0.894348
[3472]	train-logloss:0.044625	train-auc:0.964387	valid-logloss:0.149526	valid-auc:0.894347
[3473]	train-logloss:0.044622	train-auc:0.96439	valid-logloss:0.149515	valid-auc:0.894344
[3474]	train-logloss:0.044621	train-auc:0.964395	valid-logloss:0.149506	valid-auc:0.894346
[3475]	train-logloss:0.044619	train-auc:0.964398	valid-logloss:0.149512	valid-auc:0.894342
[3476]	train-logloss:0.044616	train-auc:0.964401	valid-logloss:0.149501	valid-auc:0.894393
[3477]	train-logloss:0.044614	train-auc:0.964406	valid-logloss:0.149502	valid-auc:0.8944
[3478]	train-logloss:0.044612	train-auc:0.964411	valid-logloss:0.149504	valid-auc:0.894394
[3479]

KeyboardInterrupt: 

In [20]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 200
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9,
          #"metric": 'auc'
          }
for i in range(0,9,2):
    params['seed'] = i
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    # print(day_target)
    train = data[data.Day.isin(day_target)]
    
    dtrain = lgbm.Dataset(train.drop(['id','date','label','Day','cate'],axis=1).values, train.label.values)
    dvalid = lgbm.Dataset(valid.drop(['id','date','label','Day','cate'],axis=1).values, valid.label.values, reference=dtrain)
    bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid,\
                     feval=my_score2,\
                     verbose_eval=100,
                     early_stopping_rounds=100)
    # make the predictions
#     cv_pred += bst.predict(test.drop(['id','date','Day'],axis=1).values, num_iteration=bst.best_iteration)
    valid_pred += bst.predict(valid.drop(['id','date','label','Day'],axis=1).values, num_iteration=bst.best_iteration)

valid_pred /= 5    
# cv_pred /= 5

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

# pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0009.csv', index=False)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.110506	valid_0's score: 0.215695
[200]	valid_0's binary_logloss: 0.107953	valid_0's score: 0.246487
[300]	valid_0's binary_logloss: 0.10763	valid_0's score: 0.245516
Early stopping, best iteration is:
[227]	valid_0's binary_logloss: 0.107736	valid_0's score: 0.250822
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.110789	valid_0's score: 0.213602
[200]	valid_0's binary_logloss: 0.109406	valid_0's score: 0.217638
[300]	valid_0's binary_logloss: 0.108724	valid_0's score: 0.222123
[400]	valid_0's binary_logloss: 0.108285	valid_0's score: 0.223842
[500]	valid_0's binary_logloss: 0.108128	valid_0's score: 0.221898
Early stopping, best iteration is:
[450]	valid_0's binary_logloss: 0.108121	valid_0's score: 0.22728
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.110635	valid_0's score: 0.201644
[200]	

In [15]:
valid_pred_df = pd.DataFrame({'id': valid.id, 'score': valid_pred, 'label':  valid.label})
valid_pred_df = valid_pred_df.merge(train_cate, how='inner', on='id')
valid_pred_df.head()

,id,score,label,cate
0,e8c3a7fbd718708534a436d217a76c86ac2e0683b65ae1...,0.000091,0,7
1,d89c7b05241c510b3b24371fccd7be3667099f89f4a2ec...,0.755644,1,7
2,eb9115c595b131a80f9fc50676382f72badb8c26aa2584...,0.000325,0,7
3,eef20507bbf041f0231f053f3161061bcec68dae1d7ed0...,0.000518,0,7
4,2c3976f164024e349127ba8d5a84e48dc109ec7299d6ea...,0.360241,0,7


In [16]:
percent = valid_pred_df.label.values.sum()/valid_pred_df.shape[0]

threshold = valid_pred_df.score.quantile(1-percent)
valid_pred_df['pred_label'] = valid_pred_df.score.apply(lambda x: int(x>threshold))
valid_pred_df['isCorrect'] = valid_pred_df['label']==valid_pred_df['pred_label']
valid_pred_df['isCorrect'] = valid_pred_df['isCorrect'].astype('int')
valid_pred_df.head()

,id,score,label,cate,pred_label,isCorrect
0,e8c3a7fbd718708534a436d217a76c86ac2e0683b65ae1...,0.000091,0,7,0,1
1,d89c7b05241c510b3b24371fccd7be3667099f89f4a2ec...,0.755644,1,7,0,0
2,eb9115c595b131a80f9fc50676382f72badb8c26aa2584...,0.000325,0,7,0,1
3,eef20507bbf041f0231f053f3161061bcec68dae1d7ed0...,0.000518,0,7,0,1
4,2c3976f164024e349127ba8d5a84e48dc109ec7299d6ea...,0.360241,0,7,0,1


In [18]:
# use training data with cate==7 
print('Incorrect predictions: {}'.format(valid_pred_df[valid_pred_df['isCorrect']==0].shape[0]))
print('Total predictions: {}'.format(valid_pred_df.shape[0]))
print('False negatives: {}'.format(valid_pred_df[valid_pred_df['label']==1][valid_pred_df['isCorrect']==0].shape[0]))
print('Total positives: {}'.format(valid_pred_df[valid_pred_df['label']==1].shape[0]))

Incorrect predictions: 1374
Total predictions: 24768
False negatives: 687
Total positives: 1415


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# use all training data
print('Incorrect predictions: {}'.format(valid_pred_df[valid_pred_df['isCorrect']==0].shape[0]))
print('Total predictions: {}'.format(valid_pred_df.shape[0]))
print('False negatives: {}'.format(valid_pred_df[valid_pred_df['label']==1][valid_pred_df['isCorrect']==0].shape[0]))
print('Total positives: {}'.format(valid_pred_df[valid_pred_df['label']==1].shape[0]))

### Add weights to training data

In [4]:
train_cate = pd.read_csv("obj/id_cate_train.csv")
data = data.merge(train_cate, how='inner', on='id')
data['weight'] = data.cate.apply(lambda x: 2 if x==2 or x==7 else 1)

In [5]:
# use the last 20% data as validation set
valid = data.iloc[data.shape[0]*4//5:,:]
data = data.iloc[:data.shape[0]*4//5,:]

# weight_valid = valid.weight
# weight_data = data.weight

In [9]:
test_id = test.id
cv_pred = np.zeros(len(test_id))

valid_pred = np.zeros(valid.shape[0])
#valid_id = valid.id

In [10]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.8,
          #"metric": 'auc'
          }
for i in range(0,9,2):
    params['seed'] = i
    day_target = list(set([tmp+1 for tmp in range(31)])-\
                      set([10*tmp+i+1 if 10*tmp+i+1<=31 else 10*tmp+i-9 for tmp in range(4)]+\
                          [10*tmp+i+2 if 10*tmp+i+2<=31 else 10*tmp+i-8 for tmp in range(4)]))
    # print(day_target)
    train = data[data.Day.isin(day_target)]
    
    dtrain = lgbm.Dataset(train.drop(['id','date','label','Day','weight'],axis=1).values,\
                          train.label.values, weight=train.weight.values)
    dvalid = lgbm.Dataset(valid.drop(['id','date','label','Day','weight'],axis=1).values,\
                          valid.label.values, reference=dtrain, weight=valid.weight.values)
    bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid,\
                     feval=my_score2,\
                     verbose_eval=100,
                     early_stopping_rounds=300)
    # make the predictions
    cv_pred += bst.predict(test.drop(['id','date','Day'],axis=1).values, num_iteration=bst.best_iteration)
    valid_pred += bst.predict(valid.drop(['id','date','label','Day'],axis=1).values, num_iteration=bst.best_iteration)

valid_pred /= 5    
cv_pred /= 5

print('Final score of validation is {}'.format(my_score1(valid.label.values, valid_pred)))

pd.DataFrame({'id': test_id, 'score': cv_pred}).to_csv('submission/test_b/0013.csv', index=False)

Training until validation scores don't improve for 300 rounds.
[100]	valid_0's binary_logloss: 0.0468065	valid_0's score: 0.412607
[200]	valid_0's binary_logloss: 0.0460782	valid_0's score: 0.414193
[300]	valid_0's binary_logloss: 0.0460244	valid_0's score: 0.40861
[400]	valid_0's binary_logloss: 0.0461325	valid_0's score: 0.405995
[500]	valid_0's binary_logloss: 0.0461557	valid_0's score: 0.404819
Early stopping, best iteration is:
[225]	valid_0's binary_logloss: 0.0460397	valid_0's score: 0.415604
Training until validation scores don't improve for 300 rounds.
[100]	valid_0's binary_logloss: 0.0472591	valid_0's score: 0.395093
[200]	valid_0's binary_logloss: 0.0464122	valid_0's score: 0.397825
[300]	valid_0's binary_logloss: 0.0465003	valid_0's score: 0.402028
[400]	valid_0's binary_logloss: 0.046374	valid_0's score: 0.401704
[500]	valid_0's binary_logloss: 0.0464695	valid_0's score: 0.399824
Early stopping, best iteration is:
[296]	valid_0's binary_logloss: 0.0465018	valid_0's score: